In [7]:
 pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
import cv2
import os

# Load the face detection classifier
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Function to perform face detection and return cropped face
def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    # Select the largest detected face
    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    cropped_face = img[y:y + h, x:x + w]

    return cropped_face

# Create a directory to save the dataset
data_folder = r"C:\Users\JHA\OneDrive\Desktop\recog"

# Capture video from the camera
cap = cv2.VideoCapture(0)

# Initialize counters for each individual
individual_count = {}
max_samples_per_individual = 5  # Adjust this as per your requirement

# Process video frames
while True:
    # Read frame from the camera
    ret, frame = cap.read()

    # Detect and crop face in the frame
    cropped_face = face_cropped(frame)

    if cropped_face is not None:
        # Resize and convert the cropped face to grayscale
        face = cv2.resize(cropped_face, (200, 200))
        gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Apply histogram equalization to improve contrast
#         gray_face = cv2.equalizeHist(gray_face)

        # Generate a unique image ID for each individual
        name = "Sa"  # Change this to the individual's name
        individual_count.setdefault(name, 0)
        individual_count[name] += 1
        img_id = individual_count[name]

        # Save the face image
        file_name = f"{name}.{img_id}.jpg"
        file_path = os.path.join(data_folder, file_name)
        cv2.imwrite(file_path, gray_face, [cv2.IMWRITE_JPEG_QUALITY, 100])

        # Display the cropped face with image ID
        cv2.putText(face, str(img_id), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Cropped Face", face)

    # Break the loop if 'q' is pressed or max samples reached
    if cv2.waitKey(1) == ord('q') or (individual_count and max(individual_count.values()) >= max_samples_per_individual):
        break

# Release the video capture
cap.release()
# Close all OpenCV windows
cv2.destroyAllWindows()
print("Collecting samples is completed!!!")


#In this modified code, the histogram equalization step is added before saving the face image. It improves the contrast of the grayscale face image and enhances the details for better recognition.

In [9]:
# generate_dataset()

 Create Label


In [1]:
import numpy as np
def my_label(image_name):
    name = image_name.split('.')[0]
    
    if name=="xyz":
        return np.array([1,0,0])
    else if name=="abc":
        return np.array([0,1,0])
    else if name="amini":
        return np.array([0,0,1])
    


CREATE DATA

In [2]:
import os
import cv2
from random import shuffle
from tqdm import tqdm

In [3]:
# conda install -c conda-forge dlib



SyntaxError: invalid syntax (1200072871.py, line 1)

In [1]:
def my_data():
    data = []
    for img in tqdm(os.listdir(r"C:\Users\JHA\OneDrive\Desktop\recog")):
        path=os.path.join(r"C:\Users\JHA\OneDrive\Desktop\recog",img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (200,200))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)  
    return data

 


In [17]:
# data_folder=r"C:\Users\JHA\AAAAFACEREC\data"
data = my_data()

100%|█████████████████████████████████████████████████████████████████████████████| 1185/1185 [00:02<00:00, 398.50it/s]


In [21]:
shuffle(data)

In [22]:
train_data = data[:800]
test_data = data[800:]

# Extract X_train (images) and Y_train (labels) from the training set
X_train = np.array([item[0] for item in train_data])
Y_train = np.array([item[1] for item in train_data])
X_test= np.array([item[0] for item in test_data])
Y_test= np.array([item[1] for item in test_data])

print(f"{X_train.shape}, {Y_train.shape}, {X_test.shape}, {Y_test.shape}")

(800, 200, 200), (800, 2), (385, 200, 200), (385, 2)


Creating the Model

In [24]:
#  pip install --ignore-installed --upgrade tensorflow==1.14.0

In [112]:
# test_data[0][1].shape

(3,)

In [2]:
# data[0].shape

In [3]:
# train.shape

In [4]:
# X_train.shape

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create the CNN model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)))
model.add(MaxPooling2D((2, 2)))

# Add the second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Add the third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output from the previous layer
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(64, activation='relu'))

# Add the output layer
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=16)

   

Epoch 1/10
50/50 [==============================] - 36s 703ms/step - loss: 14.3315 - accuracy: 0.8650 - val_loss: 0.0204 - val_accuracy: 0.9948
Epoch 2/10
50/50 [==============================] - 34s 692ms/step - loss: 0.0033 - accuracy: 0.9987 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 3/10
50/50 [==============================] - 35s 697ms/step - loss: 7.4306e-04 - accuracy: 1.0000 - val_loss: 6.4391e-04 - val_accuracy: 1.0000
Epoch 4/10
50/50 [==============================] - 35s 704ms/step - loss: 2.8613e-04 - accuracy: 1.0000 - val_loss: 3.5272e-04 - val_accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - 36s 718ms/step - loss: 1.6212e-04 - accuracy: 1.0000 - val_loss: 1.8495e-04 - val_accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 41s 817ms/step - loss: 9.0120e-05 - accuracy: 1.0000 - val_loss: 1.2580e-04 - val_accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 49s 977ms/step - loss: 5.7628e-05 - accuracy: 1.0000 - va

In [2]:
# from tensorflow.keras.models import load_model
# mymodel=load_model("model.h5")

In [12]:

test_image = cv2.imread(r"C:\Users\JHA\OneDrive\Desktop\recog\Subham.184.jpg", cv2.IMREAD_GRAYSCALE)
test_image = cv2.resize(test_image, (200, 200))
test_image = np.array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)
prediction = mymodel.predict(test_image)
print(prediction)

1/1 [==============================] - 0s 64ms/step
[[0.5074901 0.4925099]]


In [62]:
!pip install tensorflow

  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0


In [3]:
import cv2
import numpy as np
import openpyxl
from datetime import datetime

# Load the trained CNN model
class_labels = ['Subham', 'Saroj']

# Load the face detection classifier
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the Excel file
workbook = openpyxl.load_workbook('attendance.xlsx')
sheet = workbook.active

# Create a temporary dictionary to track the occurrence count for each label
temp = {label: 0 for label in class_labels}

# Function to perform face detection and classification
def detect_and_classify_face(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Perform face detection
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region
        face = gray[y:y+h, x:x+w]
        
        # Preprocess the face image for classification
        face = cv2.resize(face, (200, 200))
        face = face.reshape(-1, 200, 200, 1)
        
        # Make prediction using the trained model
        prediction = mymodel.predict(face)
        class_index = np.argmax(prediction)
        
        # Get the predicted class label
        class_label = class_labels[class_index]
        onehe = class_label
        
        # Get the current date and time
        current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Update the temporary count dictionary
        temp[class_label] += 1
        
        # Check if the count is zero before updating the Excel sheet
        if temp[class_label] == 1:
            # Update the Excel sheet with the student name, date, and time
            next_row = sheet.max_row + 1
            sheet.cell(row=next_row, column=1).value = onehe
            sheet.cell(row=next_row, column=2).value = current_datetime
        
        # Draw bounding box and display the predicted class label
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 250, 0), 2)
        cv2.putText(image, str(onehe), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 250, 0), 2)
    
    return image

# Capture video from the camera
cap = cv2.VideoCapture(0)

# Process video frames
while True:
    # Read frame from the camera
    ret, frame = cap.read()
    
    # Detect and classify face in the frame
    output = detect_and_classify_face(frame)
    
    # Display the output frame
    cv2.imshow("Face Detection and Classification", output)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the changes to the Excel file
workbook.save('attendance.xlsx')

# Release the video capture
cap.release()
# Close all OpenCV windows
cv2.destroyAllWindows()



1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 25ms/step


PermissionError: [Errno 13] Permission denied: 'attendance.xlsx'

1/1 [==============================] - 0s 27ms/step
